In [1]:
!jupyter nbconvert --to script BYOL.ipynb
import BYOL
import torch
import time
from pretrain_dataloader import *
from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR
from utils import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
import torch.nn.functional as F
torch.backends.cudnn.enabled = True
import copy
import wandb

[NbConvertApp] Converting notebook BYOL.ipynb to script
[NbConvertApp] Writing 6250 bytes to BYOL.py


/Users/Derek/opt/anaconda3/lib/python3.7/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/Users/Derek/Desktop/hyperbolic_byol/mobius.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  k = torch.tensor(k)
/Users/Derek/Desktop/hyperbolic_byol/mobius.py:60: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /Users/distiller/project/pytorch/aten/src/ATen/native/TensorCompare.cpp:333.)
  res = torch.where(cond, res_0, res_c)
/Users/Derek/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet 

In [2]:
args = return_default_args()
args.batch_size = 2
train_loader = prepare_cifar_train_loader(args)
online = BYOL.euclidean_BYOL_module().to(device).to(memory_format=torch.channels_last)
target = copy.deepcopy(online).to(device).to(memory_format=torch.channels_last)
fine_to_coarse = fine_to_coarse()

Files already downloaded and verified


In [3]:
params = [
            {"name": "backbone", "params": online.network.parameters()},
            {
                "name": "classifier",
                "params": online.classifier.parameters(),
                "lr": args.classifier_lr,
                "weight_decay": args.classifier_weight_decay,
            },
            {
                "name": "coarse_classifier",
                "params": online.coarse_classifier.parameters(),
                "lr": args.classifier_lr,
                "weight_decay": args.classifier_weight_decay,
            },
            {
                "name": "momentum_classifier",
                "params": target.classifier.parameters(),
                "lr": args.classifier_lr,
                "weight_decay": args.classifier_weight_decay,
            },
            {
                "name": "coarse_momentum_classifier",
                "params": target.coarse_classifier.parameters(),
                "lr": args.classifier_lr,
                "weight_decay": args.classifier_weight_decay,
            },
            {'params': online.projector.parameters()},
            {'params':online.predictor.parameters(),}
        ]


args.lr = .001
args.wd = 1.5e-5

opt = torch.optim.AdamW(params, lr = args.lr, weight_decay = args.wd)
schedule = LinearWarmupCosineAnnealingLR(
                    opt,
                    warmup_epochs= 10 * 195,
                    max_epochs= 195 * 200,
                    warmup_start_lr=3e-05,
                    eta_min=0)

In [4]:
def training_step(data, online, target, fine_to_coarse = fine_to_coarse):
    step_metrics = {}
    
    labels2 = []
    for i in data[2]:
        labels2.append(fine_to_coarse[i.item()])
    labels2 = torch.Tensor(labels2, device = device)
    labels2 = labels2.long()
    
    online1 = online(data[1][0])
    online2 = online(data[1][1])
    target1 = target.momentum_forward(data[1][0])
    target2 = target.momentum_forward(data[1][1])
    
    #byol_loss
    step_metrics["byol_loss"] = byol_loss_func(online1['p'], target2['z'])
    step_metrics["byol_loss"] += byol_loss_func(online2['p'], target1['z'])
    
    #cross entropy loss
    
    step_metrics["online_cross_entropy_loss"] = F.cross_entropy(online1['logits'], data[2], ignore_index=-1)
    step_metrics["momentum_cross_entropy_loss"] = F.cross_entropy(target1['logits'], data[2], ignore_index=-1)
    step_metrics["coarse_online_cross_entropy_loss"] = F.cross_entropy(online1['coarse_logits'], labels2, ignore_index=-1)
    step_metrics["coarse_momentum_cross_entropy_loss"] = F.cross_entropy(target1['coarse_logits'], labels2, ignore_index=-1)
    
    #accuracy of predictions
    _, predicted = torch.max(online1['logits'], 1)
    step_metrics["online_acc1"] = (predicted == data[2]).sum()
    _, predicted = torch.max(target1['logits'], 1)
    step_metrics["target_acc1"] = (predicted == data[2]).sum()
    
    _, pred = online1['logits'].topk(5)
    data[2] = data[2].unsqueeze(1).expand_as(pred)
    step_metrics["online_acc5"] = (data[2] == pred).any(dim = 1).sum()
    _, pred = target1['logits'].topk(5)
    step_metrics["target_acc5"] = (data[2] == pred).any(dim = 1).sum()
    
    #accuracy of predictions
    _, predicted = torch.max(online1['coarse_logits'], 1)
    step_metrics["coarse_online_acc1"] = (predicted == labels2).sum()
    _, predicted = torch.max(target1['coarse_logits'], 1)
    step_metrics["coarse_target_acc1"] = (predicted == labels2).sum()
    
    _, pred = online1['coarse_logits'].topk(5)
    labels2 = labels2.unsqueeze(1).expand_as(pred)
    step_metrics["coarse_online_acc5"] = (labels2 == pred).any(dim = 1).sum()
    _, pred = target1['coarse_logits'].topk(5)
    step_metrics["coarse_target_acc5"] = (labels2 == pred).any(dim = 1).sum()
                   
        
    
    
    #metrics to track
    with torch.no_grad():
        online1p_softmax = F.softmax(online1['p'], dim = 1)
        online2p_softmax = F.softmax(online2['p'], dim = 1)
        online1rep_softmax = F.softmax(online1['Representation'], dim = 1)
        online2rep_softmax = F.softmax(online2['Representation'], dim = 1)
        online1z_softmax = F.softmax(online1['z'], dim = 1)
        online2z_softmax = F.softmax(online2['z'], dim = 1)
        
        target1z_softmax = F.softmax(target1['z'], dim = 1)
        target2z_softmax = F.softmax(target1['z'], dim = 1)
        target1rep_softmax = F.softmax(target1['Representation'], dim = 1)
        target2rep_softmax = F.softmax(target2['Representation'], dim = 1)
        
        
        cross_entropy = F.cross_entropy(online1p_softmax, target2z_softmax, ignore_index=-1) + F.cross_entropy(online2p_softmax, target1z_softmax, ignore_index=-1)
        l1_dist = F.l1_loss(online1['p'], target2['z']) + F.l1_loss(online2['p'], target1['z'])
        l2_dist = F.mse_loss(online1['p'], target2['z']) + F.mse_loss(online2['p'], target1['z'])
        smooth_l1 = F.smooth_l1_loss(online1['p'], target2['z']) + F.smooth_l1_loss(online2['p'], target1['z'])
        kl_div = F.kl_div(online1p_softmax, target2z_softmax) + F.kl_div(online2p_softmax, target1z_softmax)
        
        representation_l1 = F.l1_loss(online1['Representation'], target2['Representation']) + F.l1_loss(online2['Representation'], target1['Representation'])
        representation_l2 = F.mse_loss(online1['Representation'], target2['Representation']) + F.mse_loss(online2['Representation'], target1['Representation'])
        representation_cross_entropy = F.cross_entropy(online1rep_softmax, target2rep_softmax, ignore_index=-1) + F.cross_entropy(online2rep_softmax, target1rep_softmax, ignore_index=-1)
        representation_kl = F.kl_div(online1rep_softmax, target2rep_softmax) + F.kl_div(online2rep_softmax, target1rep_softmax)
        representation_cos_sim = F.cosine_similarity(online1['Representation'], target2['Representation']) + F.cosine_similarity(online2['Representation'], target1['Representation'])

        projection_l1 = F.l1_loss(online1['z'], target2['z']) + F.l1_loss(online2['z'], target1['z'])
        projection_l2 = F.mse_loss(online1['z'], target2['z']) + F.mse_loss(online2['z'], target1['z'])
        projection_cross_entropy = F.cross_entropy(online1z_softmax, target2z_softmax, ignore_index=-1) + F.cross_entropy(online2z_softmax, target1z_softmax, ignore_index=-1)
        projection_kl = F.kl_div(online1z_softmax, target2z_softmax) + F.kl_div(online2z_softmax, target1z_softmax)
        projection_cos_sim = F.cosine_similarity(online1['z'], target2['z']) + F.cosine_similarity(online2['z'], target1['z'])

        momentum_projection_cos_sim = F.cosine_similarity(target1['z'], target2['z'])
        momentum_representation_cos_sim = F.cosine_similarity(target1['Representation'], target2['Representation'])
        momentum_representation_l2 =  F.mse_loss(target1['Representation'], target2['Representation'])
        momentum_projection_l2 =  F.mse_loss(target1['z'], target2['z'])

        online_projection_cos_sim = F.cosine_similarity(online1['z'], online2['z'])
        online_representation_cos_sim = F.cosine_similarity(online1['Representation'], online2['Representation'])
        online_representation_l2 =  F.mse_loss(online1['Representation'], online2['Representation'])
        online_projection_l2 =  F.mse_loss(online1['z'], online2['z'])
        
        online_representation_std = torch.mean(torch.std(online1['Representation']))
        online_projection_std = torch.mean(torch.std(online1['z']))
        online_prediction_std = torch.mean(torch.std(online1['p']))
        target_representation_std = torch.mean(torch.std(target1['Representation']))
        target_projection_std = torch.mean(torch.std(target1['z']))

        step_metrics.update({
            "train_feats_cross_entropy": cross_entropy,
            "train_feats_l1_dist": l1_dist,
            "train_feats_l2_dist": l2_dist,
            "train_feats_smooth_l1": smooth_l1,
            "train_feats_kl_div": kl_div,
            "representation_l1": representation_l1,
            "representation_l2": representation_l1,
            "representation_cross_entropy": representation_cross_entropy,
            "representation_kl": representation_kl,
            "representation_cos_sim": representation_cos_sim.mean(),
            "projection_l1": projection_l1,
            "projection_l2": projection_l2,
            "projection_cross_entropy": projection_cross_entropy,
            "projection_kl": projection_kl,
            "projection_cos_sim": projection_cos_sim.mean(),
            "momentum_projection_cos_sim": momentum_projection_cos_sim.mean(),
            "momentum_representation_cos_sim": momentum_representation_cos_sim.mean(),
            "momentum_representation_l2": momentum_representation_l2,
            "momentum_representation_l2": momentum_projection_l2,
            "online_projection_cos_sim": online_projection_cos_sim.mean(),
            "online_representation_cos_sim": online_representation_cos_sim.mean(),
            "online_representation_l2": online_representation_l2,
            "online_projection_l2": online_projection_l2,
            "online_representation_std": online_representation_std,
            "online_projection_std": online_projection_std,
            "online_prediction_std": online_prediction_std,
            "target_representation_std": target_representation_std,
            "target_projection_std": target_projection_std,
            })
        
        
    return step_metrics["byol_loss"] + step_metrics["online_cross_entropy_loss"] + \
            step_metrics["momentum_cross_entropy_loss"], step_metrics

In [5]:
'''log = True
name = 'BYOL - Adam Coarse'
if log:
    wandb.init(config = args.__dict__, name = name, project = 'hyperbolic_byol')'''

"log = True\nname = 'BYOL - Adam Coarse'\nif log:\n    wandb.init(config = args.__dict__, name = name, project = 'hyperbolic_byol')"

In [7]:
step = 0
scaler = torch.cuda.amp.GradScaler()

for e in range(200):
    metrics = {
        "byol_loss": 0,
        'online_cross_entropy_loss': 0,
        'momentum_cross_entropy_loss': 0,
        'coarse_online_cross_entropy_loss': 0,
        'coarse_momentum_cross_entropy_loss': 0,
        'online_acc1': 0,
        'target_acc1': 0,
        'online_acc5': 0,
        'target_acc5': 0,
        'coarse_online_acc1': 0,
        'coarse_target_acc1': 0,
        'coarse_online_acc5': 0,
        'coarse_target_acc5': 0,
        "train_feats_cross_entropy": 0,
        "train_feats_l1_dist": 0,
        "train_feats_l2_dist": 0,
        "train_feats_smooth_l1": 0,
        "train_feats_kl_div": 0,
        "representation_l1": 0,
        "representation_l2": 0,
        "representation_cross_entropy": 0,
        "representation_kl": 0,
        "representation_cos_sim": 0,
        "projection_l1": 0,
        "projection_l2": 0,
        "projection_cross_entropy": 0,
        "projection_kl": 0,
        "projection_cos_sim": 0,
        "momentum_projection_cos_sim": 0,
        "momentum_representation_cos_sim": 0,
        "momentum_representation_l2": 0,
        "momentum_representation_l2": 0,
        "online_projection_cos_sim": 0,
        "online_representation_cos_sim": 0,
        "online_representation_l2": 0,
        "online_projection_l2": 0,
        "online_representation_std": 0,
        "online_projection_std": 0,
        "online_prediction_std": 0,
        "target_representation_std": 0,
        "target_projection_std": 0,
        }
    
    
    total_loss = 0
    now = time.time()
    for i_, data in enumerate(train_loader):
        
        with torch.autocast(device):
            step += 1
            data[0] = data[0].to(device)
            data[1][0] = data[1][0].to(device).to(memory_format=torch.channels_last)
            data[1][1] = data[1][1].to(device).to(memory_format=torch.channels_last)
            data[2] = data[2].to(device)
            
            
            loss, step_metrics = training_step(data, online, target)
            opt.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(opt)
            schedule.step()
            scaler.update()
            print('here')
            update_target_params(online.network.parameters(), target.network.parameters(), .99)
            update_target_params(online.projector.parameters(), target.projector.parameters(), .99)
                
        total_loss += loss
        for key in step_metrics:
            metrics[key] += step_metrics[key]
        break
    
    print(e, round(time.time() - now, 2), total_loss.item() / (i_+1), metrics['byol_loss'].item() / (i_+1), 
          metrics['online_acc1'].item() / 50000, metrics['target_acc1'].item() / 50000)
    break
    if log:
        i_ += 1
        for key in metrics:
            if key[-4:-1] == 'acc':
                metrics[key] = metrics[key] / 50000
            else:
                metrics[key] = metrics[key]/i_
        wandb.log(metrics)
    checkpoint = {
    'online': online.state_dict(),
    'target': target.state_dict(),
    'epoch': e,
    'optimizer': opt.optim.state_dict()
    }
    torch.save(checkpoint, 'checkpoint.pt')

here
0 20.48 13.0625 3.5625 0.0 0.0


In [45]:
torch.save(online.state_dict(), name)
torch.save(target.state_dict(), 'target' + name)

'apple'

In [16]:
torch.long

torch.int64